# Schnorr Signature

In [57]:
import hashlib
import random

import galois

# Define our field
GF = galois.GF(2**8)
G = GF.primitive_element


def rand_scalar():
    return random.randint(0, GF.order - 1)


# Alice computes her secret key a and her public key A
a = rand_scalar()
A = a * G

# Picks a random nonce r and crates a commitment R
r = rand_scalar()
R = r * G

# Blinds her secret key and creates a signature s, sends (R, s)
s = a + r

# Bob verifies that the s matches Alice's public key A and the commitment R
assert s * G == A + R

This scheme is susceptible to a forgery attack:

In [58]:
# Alice uses a random signature s'
s_prime = rand_scalar()

# Forges the commitment R', sends (R', s') to Bob
R_prime = s_prime * G - A

# Bob checks (R', s')
assert s_prime * G == A + R_prime

In [59]:
%%latex

The public key $A$ cancels out, so the equation holds for any $s$

$$s\prime G \stackrel{?}{=} A+R\prime$$

becomes

$$s\prime G \stackrel{?}{=} A+R\prime = A + (s\prime G-A)$$

So how do we fix that?

The problem is that Alice picks $(R, s)$. But, Bob can influence her by issuing a challenge $c$. $c$ is a random number that instructs alice how many times to subtract the public key.

<IPython.core.display.Latex object>

In [60]:
# Alice commits to r and creates a commitment R = rG, and sends it to Bob
R = r * G

# Bob creates a random challenge c, and sends it to Alice
c = rand_scalar()

# Alice creates a signature s using the challenge c, and sends it to Bob
s = c * a + r

# Bob verifies the signature
assert s * G == c * A + R

In [61]:
%%latex

So after the inclusion of challenge $c$, our verification

$$sG \stackrel{?}{=} cA+R$$

becomes

$$(ca+r)G \stackrel{?}{=} (caG)+(rG) = (ca+r)G$$


Our protocol is now sound, which means that Bob (the verifier) can be confident that they will not be cheated by Alice (the prover).

There is one more thing we can do to improve it. Currently, our protocol is interactive but we may apply the Fiat-Shamir transformation to the challenge produced by Alice.

<IPython.core.display.Latex object>

In [62]:
def hash(msg):
    return int(hashlib.sha256(msg.tobytes()).hexdigest(), 16) % GF.order


# Alice commits to r and creates a commitment R = rG, and sends it to Bob
R = r * G

# Computes a challenge c using a hash function
c = hash(R)

# Creates a signature s using the challenge c, and sends (R, s) to Bob
s = c * a + r

# Bob compute challenge c' using R
c_prime = hash(R)

# Verifies the signature
assert s * G == c_prime * A + R